# Advanced Topic: Using External C++ Functions

This is based on the relevant portion of the CmdStan documentation [here](https://mc-stan.org/docs/cmdstan-guide/using-external-cpp-code.html)

Consider the following Stan model, based on the bernoulli example.

In [1]:
import os
try:
    os.remove('bernoulli_external')
except:
    pass

In [2]:
from cmdstanpy import CmdStanModel
model_external = CmdStanModel(stan_file='bernoulli_external.stan', compile=False)
print(model_external.code())

functions {
  real make_odds(real theta);
}
data {
  int<lower=0> N;
  array[N] int<lower=0, upper=1> y;
}
parameters {
  real<lower=0, upper=1> theta;
}
model {
  theta ~ beta(1, 1); // uniform prior on interval 0, 1
  y ~ bernoulli(theta);
}
generated quantities {
  real odds;
  odds = make_odds(theta);
} 



As you can see, it features a function declaration for `make_odds`, but no definition. If we try to compile this, we will get an error. 

In [3]:
model_external.compile()

16:02:06 - cmdstanpy - INFO - compiling stan file /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan to exe file /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external


ValueError: Failed to compile Stan model '/home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan'. Console:

--- Translating Stan model to C++ code ---
bin/stanc  --o=/home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.hpp /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan
Semantic error in '/home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan', line 2, column 7 to column 16:
   -------------------------------------------------
     1:  functions {
     2:    real make_odds(real theta);
                ^
     3:  }
     4:  data {
   -------------------------------------------------

Function 'make_odds' is declared without specifying a definition.
make: *** [make/program:50: /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.hpp] Error 1

Command ['make', '/home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external']
	error during process

Even enabling the `--allow-undefined` flag to stanc3 will not allow this model to be compiled quite yet.

In [4]:
model_external.compile(stanc_options={'allow-undefined':True})

16:02:10 - cmdstanpy - INFO - compiling stan file /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan to exe file /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external


ValueError: Failed to compile Stan model '/home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan'. Console:

--- Translating Stan model to C++ code ---
bin/stanc --allow-undefined --o=/home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.hpp /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.4.0 -I stan/lib/stan_math/lib/boost_1.78.0 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/stan_math/lib/sundials_6.1.1/src/sundials    -DBOOST_DISABLE_ASSERTS          -c -Wno-ignored-attributes   -include /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/user_header.hpp -x c++ -o /home/brian/Dev/py/cmdstanpy

To resolve this, we need to both tell the Stan compiler an undefined function is okay **and** let C++ know what it should be. 

We can provide a definition in a C++ header file by using the `user_header` argument to either the CmdStanModel constructor or the `compile` method. 

This will enables the `allow-undefined` flag automatically.

In [5]:
model_external.compile(user_header='make_odds.hpp')
assert model_external.exe_file is not None

16:03:06 - cmdstanpy - INFO - compiling stan file /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external.stan to exe file /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external
16:03:25 - cmdstanpy - INFO - compiled model executable: /home/brian/Dev/py/cmdstanpy/docsrc/users-guide/examples/bernoulli_external


We can then run this model and inspect the output

In [6]:
fit = model_external.sample(data={'N':10, 'y':[0,1,0,0,0,0,0,0,0,1]})
fit.stan_variable('odds')

16:03:31 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status


chain 1 |██████████| 00:00 Sampling completed
chain 2 |██████████| 00:00 Sampling completed
chain 3 |██████████| 00:00 Sampling completed
chain 4 |██████████| 00:00 Sampling completed


16:03:31 - cmdstanpy - INFO - CmdStan done processing.


array([0.28554 , 0.395692, 0.38404 , ..., 0.581201, 0.406603, 0.244602])

The contents of this header file are presented without comment:

```c++
#include <ostream>

double make_odds(const double& theta, std::ostream *pstream__) {
  return theta / (1 - theta);
}
```

Additional guidance, including on writing functions with known derivatives, can be found in the [CmdStan documentation](https://mc-stan.org/docs/cmdstan-guide/using-external-cpp-code.html).